In [1]:
import os
import sys

sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"
from quantfreedom.class_practice.base import backtest_df_only
from quantfreedom.class_practice.enums import *
import numpy as np
import pandas as pd


np.set_printoptions(formatter={"float_kind": "{:.2f}".format})
pd.options.display.float_format = "{:,.2f}".format

price_data = pd.read_hdf("../../tests/data/4hBTCETH.hd5")
%load_ext autoreload
%autoreload 2


In [2]:
price_data

symbol                BTCUSDT                                ETHUSDT           \
candle_info              open      high       low     close     open     high   
open_time                                                                       
2023-09-11 00:00:00 25,841.60 25,876.73 25,652.78 25,740.39 1,617.42 1,619.00   
2023-09-11 04:00:00 25,740.38 25,900.69 25,720.82 25,813.65 1,612.40 1,617.96   
2023-09-11 08:00:00 25,813.65 25,830.95 25,577.30 25,601.27 1,612.22 1,612.60   
2023-09-11 12:00:00 25,601.27 25,755.09 24,951.00 25,115.97 1,586.54 1,598.34   
2023-09-11 16:00:00 25,115.98 25,250.49 24,990.38 25,020.66 1,562.64 1,568.06   
2023-09-11 20:00:00 25,020.66 25,217.00 24,901.00 25,137.00 1,539.36 1,548.29   

symbol                                 
candle_info              low    close  
open_time                              
2023-09-11 00:00:00 1,603.22 1,612.41  
2023-09-11 04:00:00 1,608.00 1,612.23  
2023-09-11 08:00:00 1,575.53 1,586.53  
2023-09-11 12:00:00 1,550.00 1,562.65  
2023-09-11 16:00:00 1,536.83 1,539.37  
2023-09-11 20:00:00 1,531.01 1,543.27

In [3]:
entries_array = np.array(
    [
        [False, False, True, True, False, False],
        [True, True, True, True, False, False],
    ]
).T
entries = pd.DataFrame(entries_array, index=price_data.index)
exit_signals = np.array(
    [
        [False, False, False, False, False, True],
        [False, False, False, False, True, False],
    ]
).T
exit_signals = pd.DataFrame(exit_signals, index=price_data.index)


In [4]:
account_state = AccountState()
backtest_settings = BacktestSettings()
exchange_settings = ExchangeSettings()
order_settings_arrays = OrderSettingsArrays(
    risk_account_pct_size=np.array([1.0]) / 100,
    sl_based_on_add_pct=np.array([0.01]) / 100,
    sl_based_on_lookback=np.array([30]),
    risk_reward=np.array([2.0, 5.0]),
    leverage_type=np.array([LeverageType.Dynamic]),
    sl_candle_body_type=np.array([CandleBodyType.Low]),
    increase_position_type=np.array([IncreasePositionType.RiskPctAccountEntrySize]),
    stop_loss_type=np.array([StopLossType.SLBasedOnCandleBody]),
    take_profit_type=np.array([TakeProfitType.RiskReward]),
    max_equity_risk_pct=np.array([100]) / 100,
    order_type=np.array([OrderType.Long]),
    sl_to_be_based_on_candle_body_type=np.array([CandleBodyType.High]),
    sl_to_be_when_pct_from_candle_body=np.array([1, 2]) / 100,
    sl_to_be_zero_or_entry=np.array([SLToBeZeroOrEntryType.ZeroLoss]),
    trail_sl_based_on_candle_body_type=np.array([CandleBodyType.Close]),
    trail_sl_when_pct_from_candle_body=np.array([2.0]) / 100,
    trail_sl_by_pct=np.array([1.0]) / 100,
    static_leverage=np.array([1.0]),
    tp_fee_type=np.array([TakeProfitFeeType.Limit])
)


In [5]:
test = 'hello'
print(f"yes yes yes yes {test}")

yes yes yes yes hello


In [13]:
backtest_df_only(
    account_state=account_state,
    order_settings_arrays=order_settings_arrays,
    backtest_settings=backtest_settings,
    exchange_settings=exchange_settings,
    price_data=price_data,
    entries=entries,
)

Creating cartesian product ... after this the backtest will start, I promise :).

Starting the backtest now ... and also here are some stats for your backtest.

Total symbols: 2
Total indicator settings per symbol: 1
Total indicator settings to test: 2
Total order settings per symbol: 4
Total order settings to test: 8
Total candles per symbol: 6
Total candles to test: 48

Total combinations to test: 8

New Symbol

New Indicator Setting

New Order Setting

Checking Next Bar for entry or exit

Checking Next Bar for entry or exit

Checking Next Bar for entry or exit
Long Order - Calculate Stop Loss - calculate_stop_loss
Long Order - Candle Body Getter - __get_candle_body_price_low
Long Order - Calculate Stop Loss - sl_price= 25574.0
Long Order - Increase Position - __get_possible_loss= 10.0
Long Order - Increase Position - risk_pct_of_account_and_sl_based_on_not_in_pos
Long Order - Increase Position - entry_size= 4415.9 position_size= 4415.9
Long Order - Increase Position - entry_price= 2